## Connect

In [2]:
import pypgx.pg.rdbms.graph_server as graph_server

base_url = "https://localhost:7007"
username = "graphuser"
password = "WELcome123##"

instance = graph_server.get_instance(base_url, username, password)
session = instance.create_session("jupyter")
analyst = session.create_analyst()

print(session)

PgxSession(id: 1ceae1ce-1cd1-403b-8a2e-ac31c449a7e3, name: python_pgx_client)


In [3]:
graph = session.get_graph("hr")
graph

PgxGraph(name: hr, v: 215, e: 509, directed: True, memory(Mb): 0)

## Create Graph

In [4]:
statement = '''
CREATE PROPERTY GRAPH graph2
  VERTEX TABLES (
    customer
      KEY (cst_id)
      LABEL customer
      PROPERTIES (cst_id, first_name, last_name)
  , account
      KEY (acc_id)
      LABEL account
      PROPERTIES (acc_id)
  )
  EDGE TABLES (
    transaction
      KEY (txn_id)
      SOURCE KEY(acc_id_src) REFERENCES account
      DESTINATION KEY(acc_id_dst) REFERENCES account
      LABEL transferred_to
  , account
      KEY (acc_id)
      SOURCE KEY(cst_id) REFERENCES customer
      DESTINATION KEY(acc_id) REFERENCES account
      LABEL owns
  )
'''

In [5]:
session.prepare_pgql(statement).execute()

False

In [6]:
graph = session.get_graph("GRAPH2")
graph

PgxGraph(name: Moneyflows, v: 18000, e: 309240, directed: True, memory(Mb): 17)

In [7]:
graph.query_pgql("""
SELECT a1.acc_id AS a1_acc_id, t.datetime, t.amount, a2.acc_id AS a2_acc_id
FROM MATCH (a1)-[t:transferred_to]->(a2)
LIMIT 5
""").print()

+------------------------------------------------------+
| a1_acc_id | datetime            | amount | a2_acc_id |
+------------------------------------------------------+
| 8001      | 2020-11-15 01:41:55 | 700.0  | 1160      |
| 8001      | 2020-11-14 08:59:55 | 400.0  | 1398      |
| 8001      | 2020-11-19 15:21:33 | 300.0  | 1398      |
| 8001      | 2020-11-13 19:38:21 | 200.0  | 1398      |
| 8001      | 2020-11-16 12:48:39 | 900.0  | 1642      |
+------------------------------------------------------+



In [8]:
graph.publish()